# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Признаки
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* NotRepaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

Целевой признак
* Price — цена (евро)


## Подготовка данных

Набор данных находится в файле autos.csv

Загрузим и изучим набор данных.

In [78]:
# Импортирование необходимых модулей и атрибутов
import pandas as pd
import numpy as np
import os
import timeit
import warnings
import math
import lightgbm

from timeit import default_timer as timer
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedKFold
from numpy.random import RandomState

warnings.simplefilter(action='ignore', category=FutureWarning)

In [79]:
# Создадим вспомогательные переменные, константы
SEED = 12345

In [80]:
# Объявим функцию, которая будет читать файлы
def pth_load(pth1, pth2):
    """Sapport using os.path.exists. Load local file in primarily

    :param pth1: local addres of file
    :type pth1: object
    :param pth2: external addres of file
    :type pth2: object
    
    :raises ValueError: if file not found in addresses
    
    :rtype: DataFrame
    :return: foundly file in the form of DataFrame
    """
    if os.path.exists(pth1):
        df = pd.read_csv(pth1)
    elif os.path.exists(pth2):
        df = pd.read_csv(pth2)
    else:
        print('Something is wrong')
    return df

# Прочитаем файл, сохраним данные
df = pth_load('autos.csv', '/datasets/autos.csv')

In [81]:
# Вызовем метод 'info()' и напечатаем пять случайных строк таблицы
df.info()
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
11231,2016-03-06 12:41:20,5300,wagon,2009,manual,109,2_reihe,100000,1,gasoline,peugeot,no,2016-03-06 00:00:00,0,10409,2016-03-29 19:45:00
207024,2016-03-17 13:48:44,2800,small,1999,manual,105,golf,90000,12,petrol,volkswagen,no,2016-03-17 00:00:00,0,14558,2016-03-17 13:48:44
349975,2016-03-29 13:38:04,400,small,1996,manual,75,fiesta,150000,1,petrol,ford,NaN,2016-03-29 00:00:00,0,66793,2016-04-02 00:44:40
90946,2016-03-14 22:51:02,1500,coupe,2002,manual,131,ibiza,150000,3,gasoline,seat,no,2016-03-14 00:00:00,0,82178,2016-03-16 16:15:28
287635,2016-03-10 11:37:39,1050,small,1999,manual,65,corsa,150000,1,petrol,opel,no,2016-03-10 00:00:00,0,83257,2016-03-11 11:15:17


In [82]:
# Приведем названия столбцов к нижнему регистру
df.columns = map(str.lower, df.columns)

# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354369 non-null  object
 1   price              354369 non-null  int64 
 2   vehicletype        316879 non-null  object
 3   registrationyear   354369 non-null  int64 
 4   gearbox            334536 non-null  object
 5   power              354369 non-null  int64 
 6   model              334664 non-null  object
 7   kilometer          354369 non-null  int64 
 8   registrationmonth  354369 non-null  int64 
 9   fueltype           321474 non-null  object
 10  brand              354369 non-null  object
 11  notrepaired        283215 non-null  object
 12  datecreated        354369 non-null  object
 13  numberofpictures   354369 non-null  int64 
 14  postalcode         354369 non-null  int64 
 15  lastseen           354369 non-null  object
dtypes: int64(7), object(

Проверим пул значений в столбце *numberofpictures*

In [83]:
df['numberofpictures'].unique()

array([0], dtype=int64)

Все значения одинаковые, поэтому уберем этот столбец. Столбец *postalcode* не поможет в определении стоимости автомобиля, удалим его тоже. 

In [84]:
# Удаляем столбец 'postalcode' и 'numberofpictures'
df = df.drop(['postalcode', 'numberofpictures'], axis=1)

# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354369 non-null  object
 1   price              354369 non-null  int64 
 2   vehicletype        316879 non-null  object
 3   registrationyear   354369 non-null  int64 
 4   gearbox            334536 non-null  object
 5   power              354369 non-null  int64 
 6   model              334664 non-null  object
 7   kilometer          354369 non-null  int64 
 8   registrationmonth  354369 non-null  int64 
 9   fueltype           321474 non-null  object
 10  brand              354369 non-null  object
 11  notrepaired        283215 non-null  object
 12  datecreated        354369 non-null  object
 13  lastseen           354369 non-null  object
dtypes: int64(5), object(9)
memory usage: 37.9+ MB


Проверим пул значений в столбце *registrationmonth*

In [85]:
df['registrationmonth'].unique()

array([ 0,  5,  8,  6,  7, 10, 12, 11,  2,  3,  1,  4,  9], dtype=int64)

Есть значения "0", видимо заглушка при отсутствии в анкете. Оставим как есть, едва ли это имеет большую корреляцию на цену, ведь это месяц регистрации, а не продажи.



In [86]:
# Напечатаем количество дубликатов и выведем сами строки
print('Дубликатов в df:', df.duplicated().sum())
df[df.duplicated(keep=False)].sort_values('datecrawled')

Дубликатов в df: 5


,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,lastseen
18311,2016-03-07 12:00:46,10000,wagon,2013,manual,184,golf,60000,7,gasoline,volkswagen,no,2016-03-07 00:00:00,2016-03-20 12:49:27
149164,2016-03-07 12:00:46,10000,wagon,2013,manual,184,golf,60000,7,gasoline,volkswagen,no,2016-03-07 00:00:00,2016-03-20 12:49:27
88087,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,2016-03-09 09:46:57
171088,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,2016-03-09 09:46:57
41529,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,2016-03-18 18:46:15
325651,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,2016-03-18 18:46:15
90964,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,2016-03-28 08:46:21
231258,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,2016-03-28 08:46:21
187735,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,2016-04-07 09:44:54
258109,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,2016-04-07 09:44:54


Количество не большое, но все же удалим дубликаты

In [87]:
# Удалим полные дубликаты
df = df.drop_duplicates().reset_index(drop=True)

# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354364 entries, 0 to 354363
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354364 non-null  object
 1   price              354364 non-null  int64 
 2   vehicletype        316874 non-null  object
 3   registrationyear   354364 non-null  int64 
 4   gearbox            334531 non-null  object
 5   power              354364 non-null  int64 
 6   model              334659 non-null  object
 7   kilometer          354364 non-null  int64 
 8   registrationmonth  354364 non-null  int64 
 9   fueltype           321469 non-null  object
 10  brand              354364 non-null  object
 11  notrepaired        283210 non-null  object
 12  datecreated        354364 non-null  object
 13  lastseen           354364 non-null  object
dtypes: int64(5), object(9)
memory usage: 37.9+ MB


Основная проблема заключается в том, что во многих строках отсутствует модель автомобиля. При этом бренд заполнен во всех строках. Есть варинты заполнить пропуски в рамках бренда самой популярной моделью, при этом учитывать мощность, которая также везде присутствует (хотя есть нулевые значения, но в это вмешиваться не будем, вдруг это связано с отсутствием двигателя). 

Если не получится найти ни одной модели в данном бренде и мощности, то заполним самой популярной моделью в бренде.

In [88]:
# Заполним пропуски моделей машин по условиям, описанным выше
for brand in df['brand'].unique():
    for power in df[df['brand'] == brand]['power'].unique():
        filter_1 = df['brand'] == brand
        filter_2 = df['power'] == power
        try:
            df.loc[filter_1 & filter_2, 'model'] = df[filter_1 & filter_2]['model'].fillna(df[filter_1 & filter_2]['model'].mode()[0])
        except:
            try:
                df.loc[filter_1 & filter_2, 'model'] = df[filter_1]['model'].fillna(df[filter_1]['model'].mode()[0])
            except:
                continue
                
# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354364 entries, 0 to 354363
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354364 non-null  object
 1   price              354364 non-null  int64 
 2   vehicletype        316874 non-null  object
 3   registrationyear   354364 non-null  int64 
 4   gearbox            334531 non-null  object
 5   power              354364 non-null  int64 
 6   model              350990 non-null  object
 7   kilometer          354364 non-null  int64 
 8   registrationmonth  354364 non-null  int64 
 9   fueltype           321469 non-null  object
 10  brand              354364 non-null  object
 11  notrepaired        283210 non-null  object
 12  datecreated        354364 non-null  object
 13  lastseen           354364 non-null  object
dtypes: int64(5), object(9)
memory usage: 37.9+ MB


Посмотрим, какие модели машин остались без марки

In [89]:
df[df['model'].isnull()]['brand'].unique()

array(['sonstige_autos'], dtype=object)

На самом деле *sonstige_autos* - это подержанные машины, по какой-то причине эта строка попадает в столбец брэнда. Мы можем заполнить марку таких машин словом *sonstige*, чтобы не оставлять пропусков.

In [90]:
# Заполним пропуски моделей по условиям, описанным выше
df.loc[df['brand'] == 'sonstige_autos', 'model'] = df[df['brand'] == 'sonstige_autos']['model'].fillna('sonstige')

# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354364 entries, 0 to 354363
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354364 non-null  object
 1   price              354364 non-null  int64 
 2   vehicletype        316874 non-null  object
 3   registrationyear   354364 non-null  int64 
 4   gearbox            334531 non-null  object
 5   power              354364 non-null  int64 
 6   model              354364 non-null  object
 7   kilometer          354364 non-null  int64 
 8   registrationmonth  354364 non-null  int64 
 9   fueltype           321469 non-null  object
 10  brand              354364 non-null  object
 11  notrepaired        283210 non-null  object
 12  datecreated        354364 non-null  object
 13  lastseen           354364 non-null  object
dtypes: int64(5), object(9)
memory usage: 37.9+ MB


In [91]:
# Все модели 'other' заменим конструкцией 'брэнд_other'
other_models = pd.DataFrame()
other_models['brand'] = df['brand']
other_models['other'] = '_other'
other_models['sum'] = other_models['brand'] + other_models['other']
df.loc[df['model'] == 'other','model'] = np.nan
df.loc[df['model'].isnull(), 'model'] = df['model'].fillna(other_models['sum'])

# Проверим результат
for brand in df['brand'].unique():
    print(df[df['brand'] == brand]['model'].unique())

['golf' 'volkswagen_other' 'passat' 'polo' 'scirocco' 'transporter'
 'jetta' 'eos' 'touran' 'lupo' 'caddy' 'tiguan' 'sharan' 'up' 'fox'
 'beetle' 'touareg' 'kaefer' 'phaeton' 'cc' 'bora' 'amarok']
['a4' 'a8' 'a1' 'tt' 'a6' '80' '100' 'a3' 'a2' 'a5' 'audi_other' '90' 'q7'
 'q3' '200' 'q5']
['grand' 'wrangler' 'cherokee' 'jeep_other']
['fabia' 'yeti' 'octavia' 'roomster' 'skoda_other' 'superb' 'citigo']
['3er' '5er' '1er' '7er' 'z_reihe' '6er' 'bmw_other' 'x_reihe' 'm_reihe'
 'i3']
['2_reihe' '3_reihe' '1_reihe' 'peugeot_other' '4_reihe' '5_reihe']
['c_max' 'ka' 'fiesta' 'escort' 'focus' 'mustang' 'mondeo' 's_max'
 'galaxy' 'ford_other' 'transit' 'fusion' 'kuga' 'b_max']
['3_reihe' 'mazda_other' '6_reihe' '5_reihe' 'rx_reihe' '1_reihe'
 'mx_reihe' 'cx_reihe']
['navara' 'micra' 'almera' 'nissan_other' 'primera' 'juke' 'qashqai'
 'x_trail' 'note']
['twingo' 'clio' 'kangoo' 'scenic' 'megane' 'r19' 'espace' 'modus'
 'renault_other' 'laguna']
['a_klasse' 'mercedes_benz_other' 'e_klasse' 'b_kl

In [92]:
# Посмотрим последнюю дату активности пользователей
df['lastseen'].max()

'2016-04-07 14:58:51'

Следовательно, машины не могут быть моложе 2016 года

In [93]:
# Посмотрим данные на наличие аномальных значений
lastseen_max_year = pd.to_datetime(df['lastseen'], format='%Y-%m-%d %H:%M:%S').dt.year.max()
df[(df['registrationyear'] > lastseen_max_year) | (df['registrationyear'] < 1800)]

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,lastseen
22,2016-03-23 14:52:51,2900,NaN,2018,manual,90,meriva,150000,5,petrol,opel,no,2016-03-23 00:00:00,2016-03-31 01:16:33
26,2016-03-10 19:38:18,5555,NaN,2017,manual,125,c4,125000,4,NaN,citroen,no,2016-03-10 00:00:00,2016-03-16 09:16:46
48,2016-03-25 14:40:12,7750,NaN,2017,manual,80,golf,100000,1,petrol,volkswagen,NaN,2016-03-25 00:00:00,2016-03-31 21:47:44
51,2016-03-07 18:57:08,2000,NaN,2017,manual,90,punto,150000,11,gasoline,fiat,yes,2016-03-07 00:00:00,2016-03-07 18:57:08
57,2016-03-10 20:53:19,2399,NaN,2018,manual,64,seat_other,125000,3,NaN,seat,no,2016-03-10 00:00:00,2016-03-25 10:17:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354107,2016-03-11 15:49:51,3600,NaN,2017,manual,86,transit,150000,5,gasoline,ford,NaN,2016-03-11 00:00:00,2016-03-12 05:45:02
354135,2016-03-29 16:47:29,1000,NaN,2017,manual,101,a4,150000,9,NaN,audi,NaN,2016-03-29 00:00:00,2016-04-06 02:44:27
354198,2016-03-17 00:56:26,2140,NaN,2018,manual,80,fiesta,150000,6,NaN,ford,no,2016-03-17 00:00:00,2016-03-29 15:45:04
354248,2016-03-25 09:37:59,1250,NaN,2018,NaN,0,corsa,150000,0,petrol,opel,NaN,2016-03-25 00:00:00,2016-04-06 07:46:13


In [94]:
# Заменим аномальное значение 'registrationyear' на медиану в модели
#df.loc[(df['registrationyear'] > lastseen_max_year) , (df['registrationyear'] < 1800), 'registrationyear'] = df['registrationyear'].median()
for model in df['model'].unique():
    filter_1 = df['registrationyear'] > lastseen_max_year
    filter_2 = df['registrationyear'] < 1800
    filter_3 = df['model'] == model
    df.loc[(filter_1 | filter_2) & filter_3, 'registrationyear'] = df[filter_3]['registrationyear'].median()

# Проверим результат
df[(df['registrationyear'] > lastseen_max_year) | (df['registrationyear'] < 1800)]

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,lastseen


Пустые значения в столбцах *vehicletype*, *gearbox*, *fueltype* и *notrepaired* заполним самым популярным значением модели.

In [95]:
values = ['vehicletype', 'gearbox', 'fueltype', 'notrepaired']
for value in values:
    for model in df['model'].unique():
        filter_1 = df['model'] == model
        try:
            df.loc[filter_1, value] = df[filter_1][value].fillna(df[filter_1][value].mode()[0])
        except:
            continue

# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354364 entries, 0 to 354363
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   datecrawled        354364 non-null  object 
 1   price              354364 non-null  int64  
 2   vehicletype        354364 non-null  object 
 3   registrationyear   354364 non-null  float64
 4   gearbox            354364 non-null  object 
 5   power              354364 non-null  int64  
 6   model              354364 non-null  object 
 7   kilometer          354364 non-null  int64  
 8   registrationmonth  354364 non-null  int64  
 9   fueltype           354364 non-null  object 
 10  brand              354364 non-null  object 
 11  notrepaired        354362 non-null  object 
 12  datecreated        354364 non-null  object 
 13  lastseen           354364 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 37.9+ MB


In [96]:
# Остались не заполненные строки, посмотрим на них
df[df['notrepaired'].isnull()]

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,lastseen
234293,2016-03-30 11:39:08,3800,wagon,1978.0,manual,0,serie_1,30000,0,gasoline,land_rover,NaN,2016-03-30 00:00:00,2016-03-30 11:39:08
280212,2016-04-02 10:53:15,0,wagon,1970.0,manual,0,serie_1,100000,0,petrol,land_rover,NaN,2016-04-02 00:00:00,2016-04-06 09:16:22


In [97]:
# Похоже, в модели 'serie_1' столбец 'notrepaired' нигде не заполнен, убедимся в этом
df[df['model'] == 'serie_1']

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,lastseen
234293,2016-03-30 11:39:08,3800,wagon,1978.0,manual,0,serie_1,30000,0,gasoline,land_rover,NaN,2016-03-30 00:00:00,2016-03-30 11:39:08
280212,2016-04-02 10:53:15,0,wagon,1970.0,manual,0,serie_1,100000,0,petrol,land_rover,NaN,2016-04-02 00:00:00,2016-04-06 09:16:22


In [98]:
# Заполним пропуск популярным значением по всему дата фрейму
df['notrepaired'] = df['notrepaired'].fillna(df['notrepaired'].mode()[0])

# Проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354364 entries, 0 to 354363
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   datecrawled        354364 non-null  object 
 1   price              354364 non-null  int64  
 2   vehicletype        354364 non-null  object 
 3   registrationyear   354364 non-null  float64
 4   gearbox            354364 non-null  object 
 5   power              354364 non-null  int64  
 6   model              354364 non-null  object 
 7   kilometer          354364 non-null  int64  
 8   registrationmonth  354364 non-null  int64  
 9   fueltype           354364 non-null  object 
 10  brand              354364 non-null  object 
 11  notrepaired        354364 non-null  object 
 12  datecreated        354364 non-null  object 
 13  lastseen           354364 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 37.9+ MB


Приведем столбцы к нужным типам данных. 
Из столбцов *datecrawled*, *datecreated* и *lastseen* слудует вынести год в виде типа данных *int*.

In [99]:
# Из столбцов 'datecrawled', 'datecreated', 'lastseen' выведем годы
values = ['datecrawled', 'datecreated', 'lastseen']
for value in values:
    df[value] = pd.to_datetime(df[value], format='%Y-%m-%d %H:%M:%S').dt.year
    
# Проверим результат
df.head()

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,lastseen
0,2016,480,sedan,1993.0,manual,0,golf,150000,0,petrol,volkswagen,no,2016,2016
1,2016,18300,coupe,2011.0,manual,190,a4,125000,5,gasoline,audi,yes,2016,2016
2,2016,9800,suv,2004.0,auto,163,grand,125000,8,gasoline,jeep,no,2016,2016
3,2016,1500,small,2001.0,manual,75,golf,150000,6,petrol,volkswagen,no,2016,2016
4,2016,3600,small,2008.0,manual,69,fabia,90000,7,gasoline,skoda,no,2016,2016


Проверим, есть ли среди столбцов *price* и *power* аномальные значения

In [100]:
print('Самая высокая цена:', df['price'].max())
print('Самая высокая мощность:', df['power'].max())

Самая высокая цена: 20000
Самая высокая мощность: 20000


Цена может достигать такой величины, менять целевой признак не будем. Но такие значения мощности не возможны. Самая мощная машина в мире - *Lotus Evija* с 2000 лошадиных сил. Более того, у каждой модели будут свои значения аномальной мощности. Посчитаем максимальное значение мощности в каждой модели и аномальные заменим на медиану по модели. 

In [101]:
# Выполним обработку по описанному выше принципу
for model in df['model'].unique():
    filter_1 = df['model'] == model
    power_row = pd.Series(df[filter_1]['power'].unique())
    q90,q10 = np.percentile(power_row,[80,20])
    intr_qr = q90-q10
    max_power = q90+(1.5*intr_qr)
    filter_2 = df['power'] > max_power
    df.loc[filter_1 & filter_2, 'power'] = df[filter_1]['power'].median()

In [102]:
# Проверим результат
print('Самая высокая мощность:', df['power'].max())

Самая высокая мощность: 640.0


Создадим новый столбец, в котором опишем, на сколько старой является машина. Возможно, с определенного момента машина становится раритетом, и начинает прибавлять в стоимости.

In [103]:
df['type_of_age'] = ['super_rarity' if x<1900 else 'rarity' if 1900<=x<1986 else 'old' if 1986<=x<2002 else 'new' for x in df['registrationyear']]

# Проверим результат
df.head()

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,lastseen,type_of_age
0,2016,480,sedan,1993.0,manual,0.0,golf,150000,0,petrol,volkswagen,no,2016,2016,old
1,2016,18300,coupe,2011.0,manual,190.0,a4,125000,5,gasoline,audi,yes,2016,2016,new
2,2016,9800,suv,2004.0,auto,163.0,grand,125000,8,gasoline,jeep,no,2016,2016,new
3,2016,1500,small,2001.0,manual,75.0,golf,150000,6,petrol,volkswagen,no,2016,2016,old
4,2016,3600,small,2008.0,manual,69.0,fabia,90000,7,gasoline,skoda,no,2016,2016,new


In [104]:
# Закодируем категориальные признаки методом Ordinal Encoding  
categorical = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired', 'type_of_age']
uncategorical = list(set(df.columns) - set(categorical))
encoder = OrdinalEncoder()
df_ordinal = pd.DataFrame(encoder.fit_transform(df[categorical]), columns=categorical) 
df_ordinal[uncategorical] = df[uncategorical]

# Проверим результат
df_ordinal.head()

,vehicletype,gearbox,model,fueltype,brand,notrepaired,type_of_age,datecrawled,power,price,registrationyear,datecreated,lastseen,kilometer,registrationmonth
0,4.0,1.0,127.0,6.0,38.0,0.0,1.0,2016,0.0,480,1993.0,2016,2016,150000,0
1,2.0,1.0,29.0,2.0,1.0,1.0,0.0,2016,190.0,18300,2011.0,2016,2016,125000,5
2,6.0,0.0,128.0,2.0,14.0,0.0,0.0,2016,163.0,9800,2004.0,2016,2016,125000,8
3,5.0,1.0,127.0,6.0,38.0,0.0,1.0,2016,75.0,1500,2001.0,2016,2016,150000,6
4,5.0,1.0,110.0,2.0,31.0,0.0,0.0,2016,69.0,3600,2008.0,2016,2016,90000,7


In [105]:
# Разделим данные на таргеты и фичи
target = df_ordinal['price']
features = df_ordinal.drop(['price'], axis=1)

In [106]:
# Отделим тестовую выборку
# Пайплайны будут сами при тренировке моделей разделять тренировочную выборку на тренировочную и валидационную
features_train, features_test, target_train, target_test = \
train_test_split(features, target, test_size=0.25, random_state=SEED)

# Проверим результат
print("Длина тренировочной выборки:", features_train.shape[0])
print("Длина тестовой выборки:", features_test.shape[0])
print("Сумма:", features_train.shape[0] + features_test.shape[0])

Длина тренировочной выборки: 265773
Длина тестовой выборки: 88591
Сумма: 354364


## Обучение моделей

Примечание: для оценки качества моделей будем применять метрику RMSE.

Обучим несколько простых моделей, подбирая параметры из определенного пула

In [107]:
# Создадим фрейм для сбора результатов
scores_data = pd.DataFrame()

# Создаем метрику RMSE
def rmse(actual, predict):
    """Making customized metric

    :param actual: real values of target
    :type actual: Series
    :param predict: predictions of target
    :type predict: Series
    
    :rtype: RMSE metric of predictions to real values
    :return: float64
    """
    predict = np.array(predict)
    actual = np.array(actual)

    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)

    return score

In [108]:
# С помощью Pipeline и GridSearchCV найдем лучшие параметры для модели DecisionTreeRegressor
rmse_score = make_scorer(rmse, greater_is_better = False)
cv_n = 5

pipe_DTRegressor = Pipeline([('scaler',  StandardScaler()),
            ('DTRegressor', DecisionTreeRegressor(random_state=SEED))])

grid_params_DTRegressor = [{'DTRegressor__max_depth': range (1, 13, 2),
                            'DTRegressor__min_samples_leaf': range (1, 8)
                           }]

CV_dtregressor = GridSearchCV (estimator = pipe_DTRegressor,
                               param_grid = grid_params_DTRegressor,
                               cv = cv_n,
                               scoring = rmse_score, verbose=0, return_train_score=True)

CV_dtregressor.fit(features_train, target_train)

dtr_best_params = CV_dtregressor.best_params_
dtr_best_model = CV_dtregressor.best_estimator_
dtr_score = CV_dtregressor.best_score_

In [109]:
# Вычислим RMSE модели к тестовой выборке
dtr_grid_predictions = dtr_best_model.predict(features_test)

# Выведем итоговую метрику
dtr_grid_rmse = rmse(target_test, dtr_grid_predictions)
print('Итоговое RMSE дерева решений: {:.1f}'.format(dtr_grid_rmse))

Итоговое RMSE дерева решений: 2085.1


In [110]:
# Для RandomForestRegressor используем Pipeline и RandomizedSearchCV
pipe_RFRegressor = Pipeline([('scaler',  StandardScaler()),
            ('RFRegressor', RandomForestRegressor(random_state=SEED))])

grid_params_RFRegressor = [{'RFRegressor__n_estimators': range (20, 40, 10),
                            'RFRegressor__max_depth': range (1, 13, 2)
                           }]

CV_rfregressor = RandomizedSearchCV (estimator = pipe_RFRegressor,
                               param_distributions = grid_params_RFRegressor,
                               cv = cv_n,
                               scoring = rmse_score, verbose=0, return_train_score=True)

CV_rfregressor.fit(features_train, target_train)

dfr_best_params = CV_rfregressor.best_params_
dfr_best_model = CV_rfregressor.best_estimator_
dfr_score = CV_rfregressor.best_score_

In [111]:
# Вычислим RMSE модели к тестовой выборке
dfr_grid_predictions = dfr_best_model.predict(features_test)

# Выведем итоговую метрику
dfr_grid_rmse = rmse(target_test, dfr_grid_predictions)
print('Итоговое RMSE случайного леса: {:.1f}'.format(dfr_grid_rmse))

Итоговое RMSE случайного леса: 1987.4


In [112]:
# Для LGBMRegressor используем Pipeline и RandomizedSearchCV
pipe_LGBMRegressor = Pipeline([('scaler',  StandardScaler()),
            ('LGBMRegressor', LGBMRegressor(random_state=SEED))])

grid_params_LGBMRegressor = [{'LGBMRegressor__n_estimators': range (30, 80, 10),
                            'LGBMRegressor__max_depth': range (1, 13, 2),
                            'LGBMRegressor__learning_rate': [0.1, 1.0]
                             }]

CV_lgbmregressor = RandomizedSearchCV (estimator = pipe_LGBMRegressor,
                               param_distributions = grid_params_LGBMRegressor,
                               cv = cv_n,
                               scoring = rmse_score, verbose=0, return_train_score=True)

CV_lgbmregressor.fit(features_train, target_train)

lgbmr_best_params = CV_lgbmregressor.best_params_
lgbmr_best_model = CV_lgbmregressor.best_estimator_
lgbmr_score = CV_lgbmregressor.best_score_

In [113]:
# Вычислим RMSE модели к тестовой выборке
lgbmr_grid_predictions = lgbmr_best_model.predict(features_test)

# Выведем итоговую метрику
lgbmr_grid_rmse = rmse(target_test, lgbmr_grid_predictions)
print('Итоговое RMSE градиентного бустинга: {:.1f}'.format(lgbmr_grid_rmse))

Итоговое RMSE градиентного бустинга: 1876.0


Проверим, как долго будут обучаться модели без подбора параметров

In [114]:
# Создадим список численных признаков для структуры "StandardScaler"
numeric = ['power', 'kilometer']

# Проигнорируем предупреждение
pd.options.mode.chained_assignment = None

# Попробуем использовать StandardScaler только к числовым фичам
scaler = StandardScaler()
scaler.fit(features_train[numeric]) 
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Заодно сравним результаты с дамми регрессорами

In [115]:
# Засекаем время
start_time = timer()

# Обучаем дамми регрессор
mean_dummy_regr = DummyRegressor(strategy="mean")
mean_dummy_regr.fit(features_train, target_train)

# Выводим время обучения модели
mean_dummy_fit_time = timer() - start_time
print("Время обучения модели: {:f} секунд".format(mean_dummy_fit_time))

# Засекаем время отдельно для предсказания
start_time = timer()
dummy_mean_predictions = mean_dummy_regr.predict(features_test)

# Выводим время предсказания модели
mean_dummy_prediction_time = timer() - start_time
print("Время предсказания модели: {:f} секунд".format(mean_dummy_prediction_time))

# Выведем итоговую метрику
mean_dummy_rmse = rmse(target_test, dummy_mean_predictions)
print('Итоговое RMSE по средним значениям: {:.1f}'.format(mean_dummy_rmse))

# Сохраним метрики
scores_data = scores_data.append({'Regressor':'DummyRegressor with mean strategy', 
                                  'RMSE':mean_dummy_rmse,
                                  'Fit time':mean_dummy_fit_time,
                                  'Prediction time':mean_dummy_prediction_time
                                 }, ignore_index=True)

Время обучения модели: 0.001074 секунд
Время предсказания модели: 0.000585 секунд
Итоговое RMSE по средним значениям: 4508.1


In [116]:
# Засекаем время
start_time = timer()

# Обучаем дамми регрессор
median_dummy_regr = DummyRegressor(strategy="median")
median_dummy_regr.fit(features_train, target_train)

# Выводим время обучения модели
median_dummy_fit_time = timer() - start_time
print("Время обучения модели: {:f} секунд".format(median_dummy_fit_time))

# Засекаем время отдельно для предсказания
start_time = timer()
dummy_median_predictions = median_dummy_regr.predict(features_test)

# Выводим время предсказания модели
median_dummy_prediction_time = timer() - start_time
print("Время предсказания модели: {:f} секунд".format(median_dummy_prediction_time))

# Выведем итоговую метрику
median_dummy_rmse = rmse(target_test, dummy_median_predictions)
print('Итоговое RMSE по медиане: {:.1f}'.format(median_dummy_rmse))

# Сохраним метрики
scores_data = scores_data.append({'Regressor':'DummyRegressor with median strategy', 
                                  'RMSE':median_dummy_rmse,
                                  'Fit time':median_dummy_fit_time,
                                  'Prediction time':median_dummy_prediction_time
                                 }, ignore_index=True)

Время обучения модели: 0.003572 секунд
Время предсказания модели: 0.000502 секунд
Итоговое RMSE по медиане: 4822.9


In [117]:
# Посмотрим время обучения и предсказания дерева решений
# Засекаем время
start_time = timer()

model = DecisionTreeRegressor(random_state=SEED, 
                              max_depth=dtr_best_params['DTRegressor__max_depth'],
                              min_samples_leaf=dtr_best_params['DTRegressor__min_samples_leaf']
                              ) 
model.fit(features_train, target_train)

# Выводим время обучения модели
treeregressor_fit_time = timer() - start_time
print("Время обучения модели: {:f} секунд".format(treeregressor_fit_time))

# Засекаем время отдельно для предсказания
start_time = timer()
treeregressor_predictions = model.predict(features_test)

# Выводим время обучения и предсказания модели
treeregressor_prediction_time = timer() - start_time
print("Время предсказания модели: {:f} секунд".format(treeregressor_prediction_time))

# Выведем итоговую метрику
treeregressor_rmse = rmse(target_test, treeregressor_predictions)
print('Итоговое RMSE: {:.1f}'.format(treeregressor_rmse))

# Сохраним метрики
scores_data = scores_data.append({'Regressor':'DecisionTreeRegressor', 
                                  'RMSE':treeregressor_rmse,
                                  'Fit time':treeregressor_fit_time,
                                  'Prediction time':treeregressor_prediction_time
                                 }, ignore_index=True)

Время обучения модели: 0.756224 секунд
Время предсказания модели: 0.028120 секунд
Итоговое RMSE: 2085.5


In [118]:
# Посмотрим время обучения и предсказания случайного леса
# Засекаем время
start_time = timer()

model = RandomForestRegressor(random_state=SEED, 
                              n_estimators=dfr_best_params['RFRegressor__n_estimators'],
                              max_depth=dfr_best_params['RFRegressor__max_depth']
                              ) 
model.fit(features_train, target_train)

# Выводим время обучения модели
forestregressor_fit_time = timer() - start_time
print("Время обучения модели: {:f} секунд".format(forestregressor_fit_time))

# Засекаем время отдельно для предсказания
start_time = timer()
forestregressor_predictions = model.predict(features_test)

# Выводим время обучения и предсказания модели
forestregressor_prediction_time = timer() - start_time
print("Время предсказания модели: {:f} секунд".format(forestregressor_prediction_time))

# Выведем итоговую метрику
forestregressor_rmse = rmse(target_test, forestregressor_predictions)
print('Итоговое RMSE: {:.1f}'.format(forestregressor_rmse))
      
# Сохраним метрики
scores_data = scores_data.append({'Regressor':'RandomForestRegressor', 
                                  'RMSE':forestregressor_rmse,
                                  'Fit time':forestregressor_fit_time,
                                  'Prediction time':forestregressor_prediction_time
                                 }, ignore_index=True)

Время обучения модели: 16.285567 секунд
Время предсказания модели: 0.283474 секунд
Итоговое RMSE: 1987.4


Использование StandardScaler только к числовым фичам не улучшило метрику по сравнению с пайплайном, который использовал StandardScaler ко всем фичам.

Используем библиотеку LightGBM для предсказания стоимости машин градиентным бустингом.

In [119]:
# Посмотрим время обучения и предсказания градиентного бустинга
# Засекаем время
start_time = timer()

model = LGBMRegressor(max_depth=lgbmr_best_params['LGBMRegressor__max_depth'], 
                      n_estimators=lgbmr_best_params['LGBMRegressor__n_estimators'],
                      learning_rate=lgbmr_best_params['LGBMRegressor__learning_rate']
                     )
model.fit(features_train, target_train)

# Выводим время обучения модели
lgbmregressor_fit_time = timer() - start_time
print("Время обучения модели: {:f} секунд".format(lgbmregressor_fit_time))

# Засекаем время отдельно для предсказания
start_time = timer()
lgbmregressor_predictions = model.predict(features_test)

# Выводим время обучения и предсказания модели
lgbmregressor_prediction_time = timer() - start_time
print("Время предсказания модели: {:f} секунд".format(lgbmregressor_prediction_time))

# Выведем итоговую метрику
lgbmregressor_rmse = rmse(target_test, lgbmregressor_predictions)
print('Итоговое RMSE: {:.1f}'.format(lgbmregressor_rmse))
      
# Сохраним метрики
scores_data = scores_data.append({'Regressor':'LGBMRegressor', 
                                  'RMSE':lgbmregressor_rmse,
                                  'Fit time':lgbmregressor_fit_time,
                                  'Prediction time':lgbmregressor_prediction_time
                                 }, ignore_index=True)

Время обучения модели: 0.643030 секунд
Время предсказания модели: 0.124477 секунд
Итоговое RMSE: 1871.4


## Анализ моделей

Посмотрим, какие результаты нам удалось получить

In [120]:
scores_data = scores_data.set_index('Regressor')
scores_data

,Fit time,Prediction time,RMSE
Regressor,,,
DummyRegressor with mean strategy,0.001074,0.000585,4508.062120
DummyRegressor with median strategy,0.003572,0.000502,4822.930404
DecisionTreeRegressor,0.756224,0.028120,2085.533985
RandomForestRegressor,16.285567,0.283474,1987.431740
LGBMRegressor,0.643030,0.124477,1871.350574


Скорость дамми регрессоров рассматривать не будем. Обучение градиентного бустинга библиотеки LGBMRegressor происходит быстрее всего, кроме того, модель получила лучшую RMSE метрику. Вторая по скорости и третья метрике RMSE модель дерева решений, случайный лес на третьем месте по скорости, но на втором по метрике. Предсказания быстрее всего делает дерево решений, на втором месте градиентный бустинг, случайный лес на третьем месте.

Исходя из параметров скорость-качества, лучше моделью в данном случае будет градиентный бустинг. Он самый точный и достаточно быстро предсказывает цену автомобиля, а также его можно быстро переобучить по новым данным. 

В несколько раз быстрее предсказывает дерево решений, если для бизнеса это важнее, чем качество, то можно выбрать эту модель.